<a href="https://colab.research.google.com/github/NavaneethKrishnanB/FacExp/blob/master/facExp_Inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Mon Mar  8 09:41:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install tensorflow-gpu

     |████████████████████████████████| 394.3MB 40kB/s 


In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

AttributeError: ignored

In [ ]:
from tensorflow.keras.layers import Lambda,Dense,Flatten,Input
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
import numpy as np
from glob import glob

In [ ]:
IMAGE_SIZE=[224,224]
train_path="/content/drive/MyDrive/images/train"
test_path="/content/drive/MyDrive/images/validation"

In [ ]:
inception=InceptionV3(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)

In [ ]:
for layer in inception.layers:
  layer.trainable=False

In [ ]:
folders=glob(train_path+"/*")
print(len(folders))

5


In [ ]:
x=Flatten()(inception.output)
prediction=Dense(len(folders),activation='softmax')(x)

In [ ]:
model=Model(inputs=inception.input,outputs=prediction)

In [ ]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, 111, 111, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, 111, 111, 32) 96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 111, 111, 32) 0           batch_normalization_188[0][0]    
____________________________________________________________________________________________

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 rotation_range=0.4,
                                 horizontal_flip=True,
                                 vertical_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
train_set=train_datagen.flow_from_directory(train_path,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
    )

Found 25190 images belonging to 5 classes.


In [ ]:
test_set=test_datagen.flow_from_directory(test_path,
                                          target_size=(224,224),
                                          batch_size=32,
                                          class_mode='categorical',
                                          shuffle=True
)

Found 6158 images belonging to 5 classes.


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

In [ ]:
checkpoint=checkpoint=ModelCheckpoint('face_exp_inception.h5',
                          monitor='val_loss',
                          mode='min',
                          save_best_only=True,
                          verbose=1)

In [ ]:
earlystop=EarlyStopping(monitor='val_loss',
                        patience=5,
                        verbose=1,
                        restore_best_weights=True)

In [ ]:
history = model.fit_generator(train_set,
                    validation_data=test_set,
                    epochs=100,
                    steps_per_epoch=(len(train_set)),
                    validation_steps=(len(test_set)),
                    callbacks=[checkpoint,earlystop]
                    )

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
788/788 [==============================] - 7597s 10s/step - loss: 4.9879 - accuracy: 0.3463 - val_loss: 5.5300 - val_accuracy: 0.3918

Epoch 00001: val_loss improved from inf to 5.53000, saving model to face_exp_inception.h5
Epoch 2/100
788/788 [==============================] - 299s 380ms/step - loss: 5.0403 - accuracy: 0.4072 - val_loss: 4.7223 - val_accuracy: 0.4191

Epoch 00002: val_loss improved from 5.53000 to 4.72227, saving model to face_exp_inception.h5
Epoch 3/100
788/788 [==============================] - 297s 377ms/step - loss: 5.0143 - accuracy: 0.4300 - val_loss: 3.8903 - val_accuracy: 0.4417

Epoch 00003: val_loss improved from 4.72227 to 3.89029, saving model to face_exp_inception.h5
Epoch 4/100
788/788 [==============================] - 294s 373ms/step - loss: 5.2441 - accuracy: 0.4268 - val_loss: 3.9927 - val_accuracy: 0.4747

Epoch 00004: val_loss did not improve from 3.89029
Epoch 5/100
788/788 [==============================] - 297s 377ms/step - loss: 4

NameError: ignored